**Init Communication**


In [ ]:
import compas_rrc as rrc
import json

from compas.geometry import Point, Vector, Frame

ros = rrc.RosClient()
ros.run()

filma = rrc.AbbClient(ros, "/rob2")
morda = rrc.AbbClient(ros, "/rob1")

connect_filma = filma.send_and_wait(rrc.PrintText("Connecting to Filma..."))
connect_morda = morda.send_and_wait(rrc.PrintText("Connecting to Morda..."))

if connect_filma == "Done" and connect_morda == "Done":
    print("Connected to both robots successfully.")
else:
    print("Failed to connect to one or both robots.")
    exit(1)

In [ ]:

def pick_and_place(robot, pick_frame, place_frame, waypoint_frame, idle_frame, speed, zone):
    approach_pick = pick_frame.z + 150
    approch_place = place_frame.z + 150
    robot.send_and_wait(rrc.MoveToFrame(frame=approach_pick, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))
    robot.send_and_wait(rrc.MoveToFrame(frame=pick_frame, speed=speed/2, zone=zone, motion_type=rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    robot.send_and_wait(rrc.SetDigital("abb_scalable_io", 0))
    robot.send_and_wait(rrc.WaitTime(1.0))
    robot.send_and_wait(rrc.SetDigital("abb_scalable_io", 1))
    robot.send_and_wait(rrc.MoveToFrame(frame=waypoint_frame, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))
    robot.send_and_wait(rrc.MoveToFrame(frame=approch_place, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))
    robot.send_and_wait(rrc.MoveToFrame(frame=place_frame, speed=speed/2, zone=zone, motion_type=rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    robot.send_and_wait(rrc.SetDigital("abb_scalable_io", 0))
    robot.send_and_wait(rrc.MoveToFrame(frame=approch_place, speed=speed, zone=zone, motion_type=rrc.Motion.LINEAR))
    task = robot.send_and_wait(rrc.MoveToFrame(frame=idle_frame, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))
    if task == "Done":
        print("Pick and place operation completed successfully.")
    else:
        print("Pick and place operation failed.")

def go_home():
    filma.send_and_wait(rrc.MoveToJoints(joints=[-20,-12,55,0,48,70],ext_axes=None,speed=200,zone=rrc.Zone.FINE))
    # morda.send_and_wait(rrc.MoveToJoints(joints=[-13,-17,55,0,48,-103],ext_axes=None,speed=200,zone=rrc.Zone.FINE))



In [ ]:
def load_frames_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    def to_compas_frame(frame_data):
        point = Point(*frame_data['position'])
        xaxis = Vector(*frame_data['x_axis'])
        yaxis = Vector(*frame_data['y_axis'])
        return Frame(point, xaxis, yaxis)

    pick_frames = [to_compas_frame(f) for f in data.get("pick_frames", [])]
    place_frames = [to_compas_frame(f) for f in data.get("place_frames", [])]
    waypoint_frame = to_compas_frame(data["waypoint_frame"])

    return pick_frames, place_frames, waypoint_frame


In [ ]:
pick_frames, place_frames, waypoint_frame = load_frames_from_json(r"D:\IAAC\Robots\compas-rrc\demo_rrc\task_filomena.json")

In [ ]:
for f in pick_frames:
    print(f)